In [1]:
from new_model import * 

In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    torch.manual_seed(seed)
    if device == 'cuda':
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
#CIFAR 10 : Data 다운로드 경로지정
batch_size = 128


#Data Process 
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

train_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR10) * 95 / 100)
num_val = len(train_CIFAR10) - num_train
train_CIFAR10, val_CIFAR10 = torch.utils.data.random_split(train_CIFAR10, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR10, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def node_prune(model, pruning_ratio):
    for layer in model.modules():
        if isinstance(layer, M_BasicBlock):
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[0][layer.M_relu1.active != 0]
            cutoff1 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu1.active.data[layer.get_2norm()[0] < cutoff1] *= 0
            
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[1][layer.M_relu2.active != 0]
            cutoff2 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu2.active.data[layer.get_2norm()[1] < cutoff2] *= 0

In [5]:
## fine_tuning
def fine_tuning(model, mode, EPOCHS, INITIAL_LR):
    # total number of training epochs
    CHECKPOINT_PATH = "./10times_CIFAR10"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)    
    for i in range(EPOCHS):
        if i % (EPOCHS/10) == 0 : #10번 자르는 거
            print(f'{int(i//(EPOCHS/10)+1)}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{int(i//(EPOCHS/10)+1)}번 자르고 난 후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 100 == 99 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

In [6]:
###8. test_model
def test_model(model):
    model.to(device)
    model.eval()

    total_examples = 0
    correct_examples = 0
    softmax = torch.nn.Softmax(dim=1)

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            pred = model(inputs)
            total_examples += inputs.shape[0]

            out = softmax(pred)
            out = torch.max(out, 1)

            correct_examples += torch.sum(targets==out[1]).cpu().data.numpy().tolist()

    avg_acc = correct_examples / total_examples
    print("Total examples is {}, correct examples is {}; Test accuracy: {}".format(total_examples, correct_examples, avg_acc))

## 모델지정

### -------------------------------------------------- ### 

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

MOMENTUM = 0.9

REG = 5e-4

criterion = nn.CrossEntropyLoss()

##5번 자를때 
# ratio = 1-0.5**0.194
##10번 자를때
ratio = 1-0.5**0.08744

## cifar10

In [8]:
lr = 0.02
basic = 2
for seed in [10, 100, 777] :
    set_seed(seed)
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## train 
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar10.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_multistepLR_lr{lr}_seed{seed}", EPOCHS=1000, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./10times_CIFAR10/28x{basic}_multistepLR_lr{lr}_seed{seed}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x2
최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 자르고 난 후 남은 node 는 : (tensor(840., device='cuda:0'), tensor(840., device='cuda:0'))
Epoch 100: progress time is 2912.89 sec
Validation loss: 0.1758, Validation accuracy: 0.9488
2번 자르고 fine tuning을 시작합니다.
2번 자르고 난 후 남은 node 는 : (tensor(788., device='cuda:0'), tensor(788., device='cuda:0'))
Epoch 200: progress time is 6095.91 sec
Validation loss: 0.1590, Validation accuracy: 0.9548
3번 자르고 fine tuning을 시작합니다.
3번 자르고 난 후 남은 node 는 : (tensor(736., device='cuda:0'), tensor(736., device='cuda:0'))
Epoch 300: progress time is 9460.95 sec
Validation loss: 0.1834, Validation accuracy: 0.9464
4번 자르고 fine tuning을 시작합니다.
4번 자르고 난 후 남은 node 는 : (tensor(688., device='cuda:0'), tensor(688., device='cuda:0'))
Epoch 400: progress time is 12540.5 sec
Validation loss: 0.2100, Validation accuracy: 0.9428
5번 자르고 fine tuning을 시작합니다.
5번 자르고 난 후 남은 node 는 : (tensor(6

# Cosine scheduler

In [9]:
## fine_tuning
def fine_tuning(model, mode, EPOCHS, INITIAL_LR):
    # total number of training epochs
    CHECKPOINT_PATH = "./10times_CIFAR10"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)    
    for i in range(EPOCHS):
        if i % (EPOCHS/10) == 0 : #10번 자르는 거
            print(f'{int(i//(EPOCHS/10)+1)}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{int(i//(EPOCHS/10)+1)}번 자르고 난 후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 100 == 99 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

# cifar10

In [10]:
lr = 0.02
basic = 2
for seed in [10, 100, 777] :
    set_seed(seed)
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## train 
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar10.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_cosine_lr{lr}_seed{seed}", EPOCHS=1000, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./10times_CIFAR10/28x{basic}_cosine_lr{lr}_seed{seed}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x2
최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 자르고 난 후 남은 node 는 : (tensor(840., device='cuda:0'), tensor(840., device='cuda:0'))
Epoch 100: progress time is 2757.04 sec
Validation loss: 0.1469, Validation accuracy: 0.9568
2번 자르고 fine tuning을 시작합니다.
2번 자르고 난 후 남은 node 는 : (tensor(788., device='cuda:0'), tensor(788., device='cuda:0'))
Epoch 200: progress time is 5504.37 sec
Validation loss: 0.4109, Validation accuracy: 0.8872
3번 자르고 fine tuning을 시작합니다.
3번 자르고 난 후 남은 node 는 : (tensor(736., device='cuda:0'), tensor(736., device='cuda:0'))
Epoch 300: progress time is 8254.81 sec
Validation loss: 0.1887, Validation accuracy: 0.9444
4번 자르고 fine tuning을 시작합니다.
4번 자르고 난 후 남은 node 는 : (tensor(688., device='cuda:0'), tensor(688., device='cuda:0'))
Epoch 400: progress time is 11003.0 sec
Validation loss: 0.3620, Validation accuracy: 0.8928
5번 자르고 fine tuning을 시작합니다.
5번 자르고 난 후 남은 node 는 : (tensor(6

## test 결과

### cifar10

In [11]:
lr = 0.02
basic = 2
for seed in [10, 100, 777] :
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## test
    checkpoint_path = f"./10times_CIFAR10/28x{basic}_multistepLR_lr{lr}_seed{seed}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   
for seed in [10, 100, 777] :
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## test
    checkpoint_path = f"./10times_CIFAR10/28x{basic}_cosine_lr{lr}_seed{seed}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x2
Total examples is 10000, correct examples is 9475; Test accuracy: 0.9475
| Wide-Resnet 28x2
Total examples is 10000, correct examples is 9459; Test accuracy: 0.9459
| Wide-Resnet 28x2
Total examples is 10000, correct examples is 9465; Test accuracy: 0.9465
| Wide-Resnet 28x2
Total examples is 10000, correct examples is 9481; Test accuracy: 0.9481
| Wide-Resnet 28x2
Total examples is 10000, correct examples is 9494; Test accuracy: 0.9494
| Wide-Resnet 28x2
Total examples is 10000, correct examples is 9491; Test accuracy: 0.9491
